In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
# with soundfile.SoundFile('data/ravdessdata/Actor_16/03-01-01-01-01-01-16.wav') as sound_file:
#         X = sound_file.read(dtype="float32")
#         sample_rate=sound_file.samplerate
#         stft=np.abs(librosa.stft(X))

In [1]:
# stft

In [2]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [4]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y,z=[],[],[]
    for file in glob.glob("data\\ravdessdata\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        # Third number (index position 2) defines the mood in the .wav file
        number = file_name.split("-")[2]
        emotion=emotions[number]
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        z.append(number)
    dataset = { 'code': z,
                'emotions': y,
                'features': x
              }
    features_df = pd.DataFrame(dataset)
    
    
    return features_df

In [5]:
features_df = load_data()
features_df.head()

,code,emotions,features
0,01,neutral,"[-693.4970092773438, 50.06439208984375, 0.5714..."
1,01,neutral,"[-683.3468017578125, 48.889259338378906, -1.68..."
2,01,neutral,"[-677.7709350585938, 51.448646545410156, 0.025..."
3,01,neutral,"[-673.9481811523438, 50.72930145263672, 1.8632..."
4,02,calm,"[-709.033447265625, 55.7661018371582, 2.698179..."


In [6]:
features_df.to_csv('data/mood_features.csv', index=False)

In [9]:
features_df.groupby('emotions').count()

,code,features
emotions,,
angry,192,192
calm,192,192
disgust,192,192
fearful,192,192
happy,192,192
neutral,96,96
sad,192,192
surprised,192,192
